Train an imitator using the best hyperparameters from multiple optuna studies. The Hyperparameters were chosen using time series splits, while this agent will be trained on the entire dataset.

Params = [n_layers: 2, activation_fn: tanh, n_units_l0: 256, dropout_l0: 0.25, n_units_l1: 368, dropout_l1: 0.45, optimizer: RMSprop, lr: 0.0005368198556846104, bs_exp: 5]

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, Subset

import pandas as pd
import os

In [11]:
BATCH = 2**5
INPUTS = 31
CLASSES = 11
DEVICE = 'cuda'
EPOCHS = 1000
SEED = 42
DATADIR = '20 bin PPO 500 results/'
SAVEDIR = DATADIR + 'surrogates/'
NAME = f'psychic_classifier_{CLASSES}'
SOC_COL = 25

In [3]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
        self.targets = torch.tensor(df.iloc[:, -1].values, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [4]:
def load_S_binned_dataset(n_bins = 10):
    #path to data directory
    data_dir = os.path.join(os.getcwd(), DATADIR) #run this from citylearn folder
    data_dir = os.path.normpath(data_dir) #resolve '..'
    #load features
    df_obs = pd.read_csv(data_dir + '/rebaseline obs.csv',
                    header=None,
                    dtype='float32')
    df_obs.set_index(df_obs.index.astype(int), inplace=True)
    #assign and bin labels
    next_soc = df_obs.iloc[1:,SOC_COL]#SOC for all but first sample
    df_obs = df_obs.iloc[:-1] #drop the last row since we have no corresponding label
    #assgin each continuous soc value to a bin
    labels = pd.cut(next_soc,
                    bins=n_bins,
                    labels=False,
                    include_lowest=True,)
    labels = labels.reset_index(drop=True)
    df_obs = df_obs.reset_index(drop=True)
    df_obs['label'] = labels
    return df_obs

In [5]:
def get_data_subset(dataset,train_ids, valid_ids, batch_size):
    """returns dataloaders with no shuffling as timeseries samples are NOT iid"""
    train_subsampler = Subset(dataset, 
                              train_ids)
    valid_subsampler = Subset(dataset, 
                              valid_ids)
    train_loader = torch.utils.data.DataLoader(train_subsampler,
                                              batch_size=batch_size,)
    valid_loader = torch.utils.data.DataLoader(valid_subsampler, 
                                             batch_size=batch_size,)
    return train_loader, valid_loader

Load training data

In [6]:
df_SA = load_S_binned_dataset(CLASSES)
sa_dataset = MyDataset(df=df_SA)
data_loader = torch.utils.data.DataLoader(sa_dataset,
                                          batch_size=BATCH,)

Def model

In [7]:
model = nn.Sequential(
    nn.Linear(INPUTS, 256),
    nn.Tanh(),
    nn.Dropout(0.25),
    nn.Linear(256,368),
    nn.Tanh(),
    nn.Dropout(0.45),
    nn.Linear(368, CLASSES)
).to(DEVICE)

In [8]:
optimizer = optim.RMSprop(model.parameters(),
                          lr=5.368e-4)

Set random seed for reproducability

In [9]:
torch.manual_seed = SEED

Train and evaluation on entire dataset

In [10]:
for epoch in range(EPOCHS):
                model.train()
                for batch_idx, (data, target) in enumerate(data_loader):

                    data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

                    optimizer.zero_grad()
                    output = model(data)
                    loss = F.cross_entropy(output, target)
                    loss.backward()
                    optimizer.step()

                # Validation of the model.
                model.eval()
                correct = 0
                with torch.no_grad():
                    for batch_idx, (data, target) in enumerate(data_loader):
                        data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                        output = model(data)
                        # Get the index of the max logit
                        pred = output.argmax(dim=1, keepdim=True)
                        correct += pred.eq(target.view_as(pred)).sum().item()

                acc_epoch = correct / len(data_loader.dataset)
                print(f'Epoch: {epoch}, Acc: {acc_epoch}')

Epoch: 0, Acc: 0.6025348253025805
Epoch: 1, Acc: 0.636789221283398
Epoch: 2, Acc: 0.6767526832610185
Epoch: 3, Acc: 0.7033569308061202
Epoch: 4, Acc: 0.7167161452386389
Epoch: 5, Acc: 0.7189997716373602
Epoch: 6, Acc: 0.7218543046357616
Epoch: 7, Acc: 0.720712491436401
Epoch: 8, Acc: 0.7325873487097511
Epoch: 9, Acc: 0.7289335464717972
Epoch: 10, Acc: 0.7213975793560173
Epoch: 11, Acc: 0.7292760904316054
Epoch: 12, Acc: 0.7210550353962092
Epoch: 13, Acc: 0.722653573875314
Epoch: 14, Acc: 0.7364695135875771
Epoch: 15, Acc: 0.7379538707467458
Epoch: 16, Acc: 0.7429778488239324
Epoch: 17, Acc: 0.7435487554236127
Epoch: 18, Acc: 0.740237497145467
Epoch: 19, Acc: 0.7290477277917333
Epoch: 20, Acc: 0.7514272664992008
Epoch: 21, Acc: 0.7538250742178579
Epoch: 22, Acc: 0.7562228819365152
Epoch: 23, Acc: 0.7581639643754282
Epoch: 24, Acc: 0.7605617720940854
Epoch: 25, Acc: 0.7631879424526148
Epoch: 26, Acc: 0.7609043160538936
Epoch: 27, Acc: 0.7668417446905686
Epoch: 28, Acc: 0.7637588490522951

In [15]:
torch.save(model, SAVEDIR + NAME + '.pth')